# SFC, FC & SC - LOBE vs HCP #

## Importing packages and setting paths ##

In [1]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np


In [2]:
#first load up the label dseg table
atlas='schaefer'

df_labels = pd.read_csv(f'resources/atlas/atlas-{atlas}_dseg.tsv',sep='\t')
df_labels

df_network_names = df_labels.networks.unique()
df_network_names

df_tabular = pd.read_csv('resources/merged_tabular.tsv',sep='\t')
#df_tabular = pd.read_csv('results/analysis/df_tabular.csv',sep='\t')

df_tabular



,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,loc_eeg_parietal,loc_eeg_temporal,loc_mri_frontal,loc_mri_occipital,loc_mri_parietal,loc_mri_temporal,loc_pet_frontal,loc_pet_occipital,loc_pet_parietal,loc_pet_temporal
0,sub-HCD0001305,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-HCD0041822,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-HCD0042420,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,sub-029,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,sub-031,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,sub-033,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,sub-035,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'




In [4]:
sfc_paths={}
sfc_paths['HCP'] = sorted(glob(sfc_pattern.format(dataset='HCP',subject='*')))
sfc_paths['LOBE'] = sorted(glob(sfc_pattern.format(dataset='LOBE',subject='*')))

fc_paths={}
fc_paths['HCP'] = sorted(glob(fc_pattern.format(dataset='HCP',subject='*')))
fc_paths['LOBE'] = sorted(glob(fc_pattern.format(dataset='LOBE',subject='*')))

sc_paths={}
sc_paths['HCP'] = sorted(glob(sc_pattern.format(dataset='HCP',subject='*')))
sc_paths['LOBE'] = sorted(glob(sc_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_paths)
#print(fc_paths)
#print(sc_paths)

In [5]:
#from snakebids.utils.snakemake_io import glob_wildcards
#subjects = glob_wildcards(sfc_pattern.format(dataset='HCP',subject='{subject}'))


In [6]:
hacky_hcp_subjects = [ path[12:26] for path in sfc_paths['HCP']]
hacky_hcp_subjects

['sub-HCD0021614', 'sub-HCD0026119', 'sub-HCD0277447']

In [7]:
hacky_lobe_subjects = [ path[13:20] for path in sfc_paths['LOBE']]
hacky_lobe_subjects


['sub-007', 'sub-009', 'sub-010']

In [8]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]
datasets=['HCP','LOBE']
for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

In [9]:
df_tabular

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,loc_eeg_parietal,loc_eeg_temporal,loc_mri_frontal,loc_mri_occipital,loc_mri_parietal,loc_mri_temporal,loc_pet_frontal,loc_pet_occipital,loc_pet_parietal,loc_pet_temporal
0,sub-HCD0001305,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-HCD0041822,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-HCD0042420,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,sub-029,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,sub-031,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,sub-033,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,sub-035,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:

#this adds RSFC features as new columns to the df_tabular

df_list=[]
for dataset in datasets:
    df_=pd.DataFrame(sfc[dataset],
                 columns=('RSFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = hacky_lobe_subjects
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

## Average FC per parcel - group difference ##

In [11]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [12]:
x1.shape

(3, 300, 300)

In [13]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [14]:
x1.shape

(3, 300)

In [15]:
x2.shape

(3, 300)

In [16]:
#add FC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = hacky_lobe_subjects
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

# Structural Connectivity #

## Average SC per parcel - group difference ##

In [17]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [18]:
x1.shape

(3, 300, 300)

In [19]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

(3, 300)

In [20]:
x1.shape

(3, 300)

In [21]:
x2.shape

(3, 300)

In [22]:
#add SC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RSC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = hacky_lobe_subjects
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [23]:
df_tabular

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,RSC_7Networks_RH_Default_PFCdPFCm_7,RSC_7Networks_RH_Default_PFCdPFCm_8,RSC_7Networks_RH_Default_PFCdPFCm_9,RSC_7Networks_RH_Default_PFCdPFCm_10,RSC_7Networks_RH_Default_pCunPCC_1,RSC_7Networks_RH_Default_pCunPCC_2,RSC_7Networks_RH_Default_pCunPCC_3,RSC_7Networks_RH_Default_pCunPCC_4,RSC_7Networks_RH_Default_pCunPCC_5,RSC_7Networks_RH_Default_pCunPCC_6
0,sub-HCD0001305,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,14.638037,8.601629,13.675084,14.696481,7.702269,5.771810,6.934359,7.493804,8.373688,4.006375
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,13.693703,12.963398,18.701582,20.530956,6.401743,7.006485,8.226523,7.101161,10.564067,3.891056
3,sub-HCD0041822,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-HCD0042420,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,sub-029,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,sub-031,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,sub-033,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,sub-035,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Network-Wise, LOBE vs HCP #

In [24]:
fc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_netbold.pconn.nii'
sc_network_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_netstruc.pconn.nii'
sfc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_netsfc.pconn.nii'


datasets=['HCP','LOBE']

In [25]:
sfc_netpaths={}
sfc_netpaths['HCP'] = sorted(glob(sfc_network_pattern.format(dataset='HCP',subject='*')))
sfc_netpaths['LOBE'] = sorted(glob(sfc_network_pattern.format(dataset='LOBE',subject='*')))

fc_netpaths={}
fc_netpaths['HCP'] = sorted(glob(fc_network_pattern.format(dataset='HCP',subject='*')))
fc_netpaths['LOBE'] = sorted(glob(fc_network_pattern.format(dataset='LOBE',subject='*')))

sc_netpaths={}
sc_netpaths['HCP'] = sorted(glob(sc_network_pattern.format(dataset='HCP',subject='*')))
sc_netpaths['LOBE'] = sorted(glob(sc_network_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_netpaths)
#print(fc_netpaths)
#print(sc_netpaths)

In [26]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_netpaths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sfc_netpaths[dataset]] ,axis=0)  
    #for path in sfc_netpaths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_netpaths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_netpaths[dataset]] ,axis=0)

## SFC ##

In [27]:
sfc['LOBE'].shape

(3, 7, 7)

### Average per Network - group difference ###

In [28]:
#set diagonal to be nan
x1 = sfc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sfc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [29]:
x1.shape

(3, 7, 7)

In [30]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [31]:
x1.shape

(3, 7)

In [32]:
x1

array([[0.04231071, 0.03165254, 0.11734484, 0.10479591, 0.04721636,
        0.08432588, 0.13196731],
       [0.14961606, 0.03344411, 0.15234053, 0.09327665, 0.11582992,
        0.12193578, 0.12769215],
       [0.01242256, 0.05526294, 0.07912111, 0.02919167, 0.02095486,
        0.05508167, 0.07081518]])

In [33]:
x2.shape

(3, 7)

In [34]:
#add SFC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NSFC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = hacky_lobe_subjects
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [35]:
print(df_tabular)

    participant_id dataset        age  sex  age_onset  ep_duration  pnes  \
0   sub-HCD0001305     HCP        NaN  NaN        NaN          NaN   NaN   
1   sub-HCD0021614     HCP   9.166667    F        NaN          NaN   NaN   
2   sub-HCD0026119     HCP  15.166667    F        NaN          NaN   NaN   
3   sub-HCD0041822     HCP        NaN  NaN        NaN          NaN   NaN   
4   sub-HCD0042420     HCP        NaN  NaN        NaN          NaN   NaN   
..             ...     ...        ...  ...        ...          ...   ...   
17         sub-029    LOBE        NaN  NaN        NaN          NaN   NaN   
18         sub-031    LOBE        NaN  NaN        NaN          NaN   NaN   
19         sub-033    LOBE        NaN  NaN        NaN          NaN   NaN   
20         sub-035    LOBE        NaN  NaN        NaN          NaN   NaN   
21         sub-038    LOBE        NaN  NaN        NaN          NaN   NaN   

   focal_seizure_type  generalized_seizures  dre  ...  \
0                 NaN         

## Functional Connectivity ##

In [36]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

### Average FC per network - group difference ###

In [37]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [38]:
x2.shape

(3, 7)

In [39]:
#add FC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NFC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = hacky_lobe_subjects
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [40]:
df_tabular

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,NSFC_Limbic,NSFC_Cont,NSFC_Default,NFC_Vis,NFC_SomMot,NFC_DorsAttn,NFC_SalVentAttn,NFC_Limbic,NFC_Cont,NFC_Default
0,sub-HCD0001305,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.047216,0.084326,0.131967,0.091186,0.150995,0.136922,0.172307,-0.000347,0.147511,0.039930
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.115830,0.121936,0.127692,0.106908,0.166418,0.136965,0.072332,0.093052,0.082952,0.052142
3,sub-HCD0041822,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-HCD0042420,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,sub-029,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,sub-031,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,sub-033,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,sub-035,LOBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Structural Connectivity ##

In [41]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

### Average SC per network - group differences ###

In [42]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [43]:
x2.shape

(3, 7)

In [44]:
#add SC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NSC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = hacky_lobe_subjects
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [45]:
print(df_tabular)

    participant_id dataset        age  sex  age_onset  ep_duration  pnes  \
0   sub-HCD0001305     HCP        NaN  NaN        NaN          NaN   NaN   
1   sub-HCD0021614     HCP   9.166667    F        NaN          NaN   NaN   
2   sub-HCD0026119     HCP  15.166667    F        NaN          NaN   NaN   
3   sub-HCD0041822     HCP        NaN  NaN        NaN          NaN   NaN   
4   sub-HCD0042420     HCP        NaN  NaN        NaN          NaN   NaN   
..             ...     ...        ...  ...        ...          ...   ...   
17         sub-029    LOBE        NaN  NaN        NaN          NaN   NaN   
18         sub-031    LOBE        NaN  NaN        NaN          NaN   NaN   
19         sub-033    LOBE        NaN  NaN        NaN          NaN   NaN   
20         sub-035    LOBE        NaN  NaN        NaN          NaN   NaN   
21         sub-038    LOBE        NaN  NaN        NaN          NaN   NaN   

   focal_seizure_type  generalized_seizures  dre  ...  NFC_Limbic  NFC_Cont  \
0       

### Save regional and network SFC, SC & FC to masterlist ###

In [46]:
df_tabular.to_csv(snakemake.output.tsv,sep='\t',index=False)


df_tabular 

NameError: name 'snakemake' is not defined